# Test the original MobileNetV2 with RGB color channel

In [1]:
%load_ext autoreload
%autoreload 2

import os, json, argparse, torch
import numpy as np
from tqdm import tqdm
from glob import glob
import torch.nn.functional as F
from multiprocessing import Pool, cpu_count

from utils import image
from utils.MobileNetV2_pretrained_imagenet import MobileNetV2
from utils.data import NumpyImageLoader
from utils.metrics import BinaryClassificationMetrics

# Initial procedure

In [2]:
# Print parameters

params = {}
params["channel"] = "RGB"
params["threshold"] = 0.500
params["test_subset"] = 5

params["patch_test_au_dir"] = "backup/MBN2-RGB/test/au"
params["patch_test_tp_dir"] = "backup/MBN2-RGB/test/tp"

params["training_log_dir"] = "backup/MBN2-RGB/checkpoints/"
MODEL_FILE = os.path.join(params["training_log_dir"], "model.ckpt")
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

params["au_subsets_file"] = "dataset/au_subsets.json"
params["tp_subsets_file"] = "dataset/tp_subsets.json"

params["casia2_au"] = "/media/antiaegis/storing/datasets/CASIA2/Au"
params["casia2_tp"] = "/media/antiaegis/storing/datasets/CASIA2/Tp"

In [3]:
def check_directories(list_dirs):
    for dir in list_dirs:
        if not os.path.exists(dir):
            print("makedirs", dir)
            os.makedirs(dir)

In [4]:
# Check directories

list_dirs = [
    params["patch_test_au_dir"],
    params["patch_test_tp_dir"],
]
check_directories(list_dirs)

# Predict features

In [5]:
# Load model

model = MobileNetV2(n_class=2, input_size=64, width_mult=1.0).to(device=DEVICE)
model.load(model_file=MODEL_FILE)
model.eval();

In [6]:
# Get list of files

with open(params["au_subsets_file"], "r") as fp:
    au_subsets = json.load(fp)
fnames_au = au_subsets[str(params["test_subset"])]
N_au = len(fnames_au)

with open(params["tp_subsets_file"], "r") as fp:
    tp_subsets = json.load(fp)
fnames_tp = tp_subsets[str(params["test_subset"])]
N_tp = len(fnames_tp)

In [ ]:
# Predict authentic patches

scores_au = []
for i in tqdm(range(N_au), total=N_au):
    # Crop patches in the image
    fname = fnames_au[i]
    file = os.path.join(params["casia2_au"], fname)
    img = image.read(file, params["channel"])

    coords, _, _ = image.slide2d(sz=img.shape[:2], K=64, S=32)
    patches = image.crop_patches(img=img, coords=coords, patch_sz=64)

    # Create dataloader of the cropped patches
    loader = NumpyImageLoader(
        ndarray_data=patches,
        batch_size=16,
        n_workers=cpu_count(),
        pin_memory=True,
        shuffle=False
    ).loader

    # Predict labels
    softmaxs = []
    for X in loader:
        X = X[0].to(DEVICE)
        logits = model(X)
        softmaxs.append(F.softmax(logits, dim=1).detach().cpu().numpy())
    softmaxs = np.concatenate(softmaxs, axis=0)

    # Save result into disk
    np.save(
        os.path.join(params["patch_test_au_dir"], fname),
        {"softmaxs": softmaxs, "coords": coords}
    )

In [ ]:
# Predict tampered images

scores_tp = []
for i in tqdm(range(N_tp), total=N_tp):
    # Crop patches in the image
    fname = fnames_tp[i]
    file = os.path.join(params["casia2_tp"], fname)
    img = image.read(file, params["channel"])

    coords, _, _ = image.slide2d(sz=img.shape[:2], K=64, S=32)
    patches = image.crop_patches(img=img, coords=coords, patch_sz=64)

    # Create dataloader of the cropped patches
    loader = NumpyImageLoader(
        ndarray_data=patches,
        batch_size=16,
        n_workers=cpu_count(),
        pin_memory=True,
        shuffle=False
    ).loader

    # Predict labels
    softmaxs = []
    for X in loader:
        X = X[0].to(DEVICE)
        logits = model(X)
        softmaxs.append(F.softmax(logits, dim=1).detach().cpu().numpy())
    softmaxs = np.concatenate(softmaxs, axis=0)

    # Save result into disk
    np.save(
        os.path.join(params["patch_test_tp_dir"], fname),
        {"softmaxs": softmaxs, "coords": coords}
    )

# Test on predicted features

In [7]:
# Create parallel pools

pools = Pool(processes=cpu_count())

In [8]:
# Get information about files on disk

au_files = glob(os.path.join(params["patch_test_au_dir"], "*.*"))
tp_files = glob(os.path.join(params["patch_test_tp_dir"], "*.*"))
n_au_files, n_tp_files = len(au_files), len(tp_files)
scores_au, scores_tp = [], []

In [9]:
# Test on authentic images

for i, file in tqdm(enumerate(au_files), total=n_au_files):
    # Load softmaxs and coords from disk
    data = np.load(file).item()
    softmaxs, coords = data["softmaxs"], data["coords"]
    softmaxs = softmaxs[:, 1]

    # Postprocess
    labels = image.post_process(softmaxs, coords, 8, params["threshold"], 32, pools=pools)
    mark = image.fusion(labels)
    scores_au.append(mark)

100%|██████████| 1253/1253 [00:05<00:00, 249.60it/s]


In [10]:
# Test on tampered images

for i, file in tqdm(enumerate(tp_files), total=n_tp_files):
    # Load softmaxs and coords from disk
    data = np.load(file).item()
    softmaxs, coords = data["softmaxs"], data["coords"]
    softmaxs = softmaxs[:, 1]

    # Postprocess
    labels = image.post_process(softmaxs, coords, 8, params["threshold"], 32, pools=pools)
    mark = image.fusion(labels)
    scores_tp.append(mark)

100%|██████████| 766/766 [00:04<00:00, 189.60it/s]


In [11]:
# Print testing metrics

metrics = BinaryClassificationMetrics()
metrics.compute_all(scores_tp, scores_au)
metrics.print_metrics()
# metrics.write_to_file(params["test_result_file"])

TP = 88.51 %; FP = 11.49 %
TN = 72.39 %; FN = 27.61 %
Accuracy = 78.50 %
Precision = 88.51 %
Recall = 66.21 %
F-score = 75.75 %


In [9]:
# Close parallel pools

pools.close()
pools.terminate()